In [1]:

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

import requests
from PIL import Image
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

!7z x '/content/drive/MyDrive/archive (3).zip'

In [3]:
model=tf.keras.Sequential([
    tf.keras.Sequential([
     tf.keras.layers.Dense(4, activation=tf.keras.activations.relu), # hidden layer 1, ReLU activation
  tf.keras.layers.Dense(4, activation=tf.keras.activations.relu), # hidden layer 2, ReLU activation
  tf.keras.layers.Dense(1, activation=tf.keras.activations.sigmoid) # ouput layer, sigmoid activation
    ])
])

In [4]:
d=pd.read_csv('/content/apple_quality.csv')
d

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,good
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,good
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,bad
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217,good
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036,good
...,...,...,...,...,...,...,...,...,...
3996,3996.0,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235285,good
3997,3997.0,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611391,bad
3998,3998.0,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229719806,good
3999,3999.0,0.278540,-1.715505,0.121217,-1.154075,1.266677,-0.776571,1.599796456,good


In [5]:
d=d.drop(d.index[-1])

In [6]:
d

,A_id,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity,Quality
0,0.0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483,good
1,1.0,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367,good
2,2.0,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473,bad
3,3.0,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217,good
4,4.0,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036,good
...,...,...,...,...,...,...,...,...,...
3995,3995.0,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784369,bad
3996,3996.0,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235285,good
3997,3997.0,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611391,bad
3998,3998.0,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229719806,good


In [7]:
y=d['Quality']


In [8]:
label=[]

In [9]:
for i in y:
  if i=='good':
    label.append(1)
  else:
     label.append(0)
label=np.array(label)
label.shape

(4000,)

one hot encoding the classes

In [10]:

d.pop('Quality')
d.pop('A_id')
d

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590483
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809367
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636473
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723217
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984036
...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784369
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235285
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611391
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229719806


In [25]:
x=d

In [12]:
d.dtypes

Size           float64
Weight         float64
Sweetness      float64
Crunchiness    float64
Juiciness      float64
Ripeness       float64
Acidity         object
dtype: object

Acidity column was creating error due to this reason

In [13]:
d['Acidity'] = d['Acidity'].apply(pd.to_numeric, errors='coerce')

In [14]:
d.dtypes

Size           float64
Weight         float64
Sweetness      float64
Crunchiness    float64
Juiciness      float64
Ripeness       float64
Acidity        float64
dtype: object

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,label,test_size=0.1,random_state=42)


In [16]:
x_train,y_train

(          Size    Weight  Sweetness  Crunchiness  Juiciness  Ripeness  \
 2957 -2.013019 -0.580230  -0.723733     1.433088  -0.324114 -0.390662   
 1096 -0.930772 -1.130286   1.215035    -0.592831  -1.575231  0.259948   
 1945  0.941632 -2.835138  -1.484719     2.567411  -2.487890 -1.269093   
 1032  0.273757 -1.723821  -0.376107     0.817061   0.992272 -0.819167   
 1018  1.932336 -1.785172  -0.050078     0.777337   0.098892  2.415391   
 ...        ...       ...        ...          ...        ...       ...   
 1130  1.203125 -1.887093  -2.551971     1.080267   3.499233 -0.427477   
 1294 -1.697843 -1.791844  -1.332840     0.106406   0.585904  2.301835   
 860   2.464079 -1.672946  -0.270204    -0.060312  -1.367107  4.491704   
 3507 -1.680704 -1.174721   1.609465     0.972546   0.358214  0.495325   
 3174 -1.720686  1.309050   0.277338     1.612230   1.674737 -1.294769   
 
        Acidity  
 2957 -1.904349  
 1096 -1.200660  
 1945 -3.168638  
 1032  1.293489  
 1018 -0.686164  
 .

In [22]:
x_train.shape

(3600, 7)

In [17]:
import numpy as np
x = np.asarray(x, dtype=np.float32)  # Convert to float32 if not already




In [18]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


In [19]:
model.fit(x_train,y_train,verbose=1,epochs=20)

Epoch 1/20
113/113 [==============================] - 2s 4ms/step - loss: 0.6815 - accuracy: 0.5833
Epoch 2/20
113/113 [==============================] - 0s 2ms/step - loss: 0.6236 - accuracy: 0.6606
Epoch 3/20
113/113 [==============================] - 0s 4ms/step - loss: 0.5801 - accuracy: 0.7089
Epoch 4/20
113/113 [==============================] - 0s 3ms/step - loss: 0.5457 - accuracy: 0.7358
Epoch 5/20
113/113 [==============================] - 0s 2ms/step - loss: 0.5178 - accuracy: 0.7519
Epoch 6/20
113/113 [==============================] - 0s 2ms/step - loss: 0.4950 - accuracy: 0.7650
Epoch 7/20
113/113 [==============================] - 0s 1ms/step - loss: 0.4774 - accuracy: 0.7714
Epoch 8/20
113/113 [==============================] - 0s 1ms/step - loss: 0.4635 - accuracy: 0.7786
Epoch 9/20
113/113 [==============================] - 0s 1ms/step - loss: 0.4532 - accuracy: 0.7850
Epoch 10/20
113/113 [==============================] - 0s 1ms/step - loss: 0.4443 - accuracy: 0.7933

In [30]:
d

,Size,Weight,Sweetness,Crunchiness,Juiciness,Ripeness,Acidity
0,-3.970049,-2.512336,5.346330,-1.012009,1.844900,0.329840,-0.491590
1,-1.195217,-2.839257,3.664059,1.588232,0.853286,0.867530,-0.722809
2,-0.292024,-1.351282,-1.738429,-0.342616,2.838636,-0.038033,2.621636
3,-0.657196,-2.271627,1.324874,-0.097875,3.637970,-3.413761,0.790723
4,1.364217,-1.296612,-0.384658,-0.553006,3.030874,-1.303849,0.501984
...,...,...,...,...,...,...,...
3995,0.059386,-1.067408,-3.714549,0.473052,1.697986,2.244055,0.137784
3996,-0.293118,1.949253,-0.204020,-0.640196,0.024523,-1.087900,1.854235
3997,-2.634515,-2.138247,-2.440461,0.657223,2.199709,4.763859,-1.334611
3998,-4.008004,-1.779337,2.366397,-0.200329,2.161435,0.214488,-2.229720


In [37]:
x_test.iloc[2].shape

(7,)

In [43]:
y[2]

'bad'

In [45]:
d1= np.expand_dims(x_test.iloc[123], axis=0)

In [46]:
f=model.predict(d1)
if np.round(f)==0:
  print("bad")
else:
  print("good")
print(f"actual : {y[2]}")

1/1 [==============================] - 0s 19ms/step
bad
actual : bad
